In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import time
import requests
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('./data/nspire_demo_deficiencies.csv', low_memory=False)

In [4]:
df

,Unit,NSPIRE Health & Safety,Deficiency Details,Question Template: Question Template ID,Inspection ID,Deficiency ID,Room/Location,Mitigation Details,Mitigation Reviewer Comment,Mitigation Late (Y/N),Location (Unit/Inside/Outside),Reviewer Comment,Reviewer Finding(s),Room,Unit Address,Status,Inspector: Full Name,Inspectable Area,H&S Determination (LT-24hr / Std 30 Day),Deficiency Location Address,Date/Time Mitigated,Deficiency Reviewed By: Full Name,Mitigation Date (PHA/POA Input),Deficiency Comments,PropertyId,NSPIRE Standards,Defect Recorded Date/Time,Date/Time Mitigation Reviewed,Date/Time Mitigation Submitted,Field Office Acknowledge (Date),Description of Repair (PHA/POA input),Corrective Timeframe,Codes,Defect Recorded Date,Shipping Address,Shipping City,Shipping Street,Shipping State/Province,Shipping Zip/Postal Code
0,THE TOWERS APARTMENTS,Severe Non-Life-Threatening,Inside -> Exit Sign -> Lighted exit sign does ...,Q-86826,INSP-01563,QR-5715325,Halls/Corridors/Stairs,Exit sign fixed,Photo submitted of mitigation. The light is on...,NaN,Inside,NaN,NaN,NaN,NaN,Mitigation Approved,NaN,Inside,Severe Non-Life-Threatening 24 hours,"913 NW 12th Street Oklahoma City, OK 73106",8/10/2022 12:30 PM,Zachary Dobelbower,NaN,Needs a battery 4th floor,0013d000008bJ27,Exit Sign,8/9/2022 7:09 PM,10/25/2022 12:07 PM,8/10/2022 10:35 AM,NaN,NaN,24 hours,"r1, r2, m2, m3",8/9/2022,"117 NW 9, OKLAHOMA CITY, OK 73102",OKLAHOMA CITY,117 NW 9,OK,73102
1,MORNINGSIDE HEIGHTS,Standard,Unit -> Windows -> Window cannot be secured ->...,Q-86231,INSP-00310,QR-4394075,Bedroom 1,NaN,NaN,NaN,Unit,NaN,NaN,NaN,UNIT: G,Acknowledged/Unmitigated,NaN,Unit,Standard 30 days,7,NaN,NaN,NaN,Damaged lock,001t000000poMNP,Windows,1/5/2022 3:25 PM,NaN,NaN,NaN,NaN,30 days,"r4, r6, m1",1/5/2022,"1516 East 6th Street, La Junta, CO 81050 Unite...",La Junta,1516 East 6th Street,CO,81050
2,CHESTNUT COURT,Standard,Unit -> Bathtub and Shower -> Bathtub componen...,Q-85787,INSP-00567,QR-4569630,Bathroom 1,NaN,NaN,NaN,Unit,NaN,NaN,NaN,352318,Acknowledged/Unmitigated,NaN,Unit,Standard 30 days,NaN,NaN,NaN,NaN,Shower not operational 2318,0013d000005B3Sr,Bathtub and Shower,3/10/2022 3:22 PM,NaN,NaN,NaN,NaN,30 days,"r3, r6, m1",3/10/2022,"Chestnut Street, Oakland, CA 94607",Oakland,Chestnut Street,CA,94607
3,MHPI VII,Standard,Unit -> Leak -> Plumbing leaks -> Failure of a...,Q-86168,INSP-00893,QR-4959929,Bathroom 1,NaN,NaN,NaN,Unit,NaN,NaN,NaN,UNIT 12: 206,Acknowledged/Unmitigated,NaN,Unit,Standard 30 days,206,NaN,NaN,NaN,Water leaking under bathroom sink,0013d000007M822,Leak,5/5/2022 9:01 PM,NaN,NaN,NaN,NaN,30 days,"r1, r2, r3, r6, m1, m4, m6",5/5/2022,"333 Market Street, Boston, MA 02135 United States",Boston,333 Market Street,MA,02135
4,VISTA VILLA,Life-Threatening,Unit -> Smoke Alarm -> Smoke alarm is not inst...,Q-86190,INSP-00993,QR-5076835,Living Room,Completed work order,NaN,NaN,Unit,NaN,NaN,NaN,13,Evidence of Mitigation Submitted,NaN,Unit,Life-Threatening 24 hours,13,5/20/2022 7:36 AM,NaN,NaN,Smoke detector missing in living room.,0013d000007SKDf,Smoke Alarm,5/18/2022 8:35 PM,NaN,7/6/2022 7:36 AM,NaN,NaN,24 hours,"r2, m2",5/18/2022,"3600 Pansy Street, Pasadena, TX 77505 United S...",Pasadena,3600 Pansy Street,TX,77505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77315,TROUBLESOME VAL-PIPPA PA,NaN,NaN,NaN,INSP-00992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"997 Kentucky 160, Hindman, KY 41822 United States",Hindman,997 Kentucky 160,KY,41822
77316,COLONEL HAMTRAMCK HOMES,NaN,NaN,NaN,INSP-03325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"12025 Dequindre Street, Hamtramck, MI 48212 Un...",Hamtramck,12025 Dequindre Street,MI,48212
77317,GRACES WAY,NaN,NaN,NaN,INSP-01296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [5]:
d = pd.read_csv('./full_fema_matches.csv')

In [20]:
m = pd.read_csv('./ph_fires_matched_2.0.csv')

In [25]:
m['total_indoor'] = m[[x for x in m.columns if 'building_fires' in x]].sum(axis=1)

In [31]:
m[m.duplicated(keep=False, subset=['total_indoor', 'corrected_address', 'corrected_unit', 'std_city', 'std_st', 'std_zip5'])].total_indoor.sum()

7671.0

In [32]:
m[m.duplicated(keep='first', subset=['total_indoor', 'corrected_address', 'corrected_unit', 'std_city', 'std_st', 'std_zip5'])].total_indoor.sum()

6732.0

In [18]:
d[d.INCIDENT_KEY == 'SC_23111_11032018_0007738_0']

,corrected_address,corrected_unit,std_city,std_st,std_zip5,INCIDENT_KEY
11308,200 CLARK ST,NaN,GREENVILLE,SC,29607,SC_23111_11032018_0007738_0
11309,200 CLARK ST,NaN,GREENVILLE,SC,29607,SC_23111_11032018_0007738_0
11310,200 CLARK ST,NaN,GREENVILLE,SC,29607,SC_23111_11032018_0007738_0
11311,200 CLARK ST,NaN,GREENVILLE,SC,29607,SC_23111_11032018_0007738_0
11312,200 CLARK ST,NaN,GREENVILLE,SC,29607,SC_23111_11032018_0007738_0
...,...,...,...,...,...,...
11371,200 CLARK ST,NaN,GREENVILLE,SC,29607,SC_23111_11032018_0007738_0
11372,200 CLARK ST,NaN,GREENVILLE,SC,29607,SC_23111_11032018_0007738_0
11373,200 CLARK ST,NaN,GREENVILLE,SC,29607,SC_23111_11032018_0007738_0
11374,200 CLARK ST,NaN,GREENVILLE,SC,29607,SC_23111_11032018_0007738_0
